In [1]:
run_checks = False

### Overview
This notebook works on the IEEE-CIS Fraud Detection competition. Here I build a simple XGBoost model based on a balanced dataset.

### Lessons:

. keep the categorical variables as single items

. Use a high max_depth for xgboost (maybe 40)


### Ideas to try:

. train divergence of expected value (eg. for TransactionAmt and distance based on the non-fraud subset (not all subset as in the case now)

. try using a temporal approach to CV

In [2]:
# all imports necessary for this notebook
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import gc
import copy
import missingno as msno 
import xgboost
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split 
from sklearn.metrics import roc_auc_score, r2_score

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/master-df-time-adjusted-top-100-v2csv/master_df_time_adjusted_top_100.csv
/kaggle/input/master-df-time-adjusted-top-100csv/master_df_time_adjusted_top_100.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-preprocessed/master_df_top_all.csv
/kaggle/input/ieee-preprocessed/master_df_top_300.csv
/kaggle/input/ieee-preprocessed/master_df_top_100.csv
/kaggle/input/ieee-preprocessed/master_df_top_200.csv


In [3]:
# Helpers
    
def seed_everything(seed=0):
    '''Seed to make all processes deterministic '''
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
def drop_correlated_cols(df, threshold, cols_to_keep, sample_frac = 1):
    '''Drops one of two dataframe's columns whose pairwise pearson's correlation is above the provided threshold'''
    if sample_frac != 1:
        dataset = df.sample(frac = sample_frac).copy()
    else:
        dataset = df
        
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        if corr_matrix.columns[i] in col_corr:
            continue
        for j in range(i):
            if corr_matrix.columns[j] in cols_to_keep:
                continue
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
    del dataset
    gc.collect()
    df.drop(columns = col_corr, inplace = True)

def calc_feature_difference(df, feature_name, indep_features, min_r2 = 0.1, min_r2_improv = 0, frac1 = 0.1,
                              max_depth_start = 2, max_depth_step = 4):
    
    from copy import deepcopy
    
    print("Feature name %s" %feature_name)
    #print("Indep_features %s" %indep_features)
    
    is_imrpoving = True
    curr_max_depth = max_depth_start
    best_r2 = float("-inf")
    clf_best = np.nan
    
    while is_imrpoving:
        clf = XGBRegressor(max_depth = curr_max_depth)

        rand_sample_indeces = df[df[feature_name].notnull()].sample(frac = frac1).index
        clf.fit(df.loc[rand_sample_indeces, indep_features], df.loc[rand_sample_indeces, feature_name]) 

        rand_sample_indeces = df[df[feature_name].notnull()].sample(frac = frac1).index
        
        pred_y = clf.predict(df.loc[rand_sample_indeces, indep_features])
        r2Score = r2_score(df.loc[rand_sample_indeces, feature_name], pred_y)
        print("%d, R2 score %.4f" % (curr_max_depth, r2Score))
        
        curr_max_depth = curr_max_depth + max_depth_step
        
        if r2Score > best_r2:
            best_r2 = r2Score
            clf_best = deepcopy(clf)
        if r2Score < best_r2 + (best_r2 * min_r2_improv) or (curr_max_depth > max_depth_start * max_depth_step and best_r2 < min_r2 / 2):
            is_imrpoving = False

    print("The best R2 score of %.4f" % ( best_r2))
    
    if best_r2 > min_r2:
        pred_feature = clf_best.predict(df.loc[:, indep_features])
        return (df[feature_name] - pred_feature)
    else:
        return df[feature_name]

In [4]:
seed_everything()
pd.set_option('display.max_columns', 500)

In [5]:
master_df = pd.read_csv('/kaggle/input/master-df-time-adjusted-top-100-v2csv/master_df_time_adjusted_top_100.csv')
master_df.head()

,TransactionID,isFraud,TransactionDT,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,P_emaildomain,R_emaildomain,C1,C2,C6,C8,C11,C13,C14,D1,D10,D15,M3,M4,M5,M6,M7,M8,M9,V12,V35,V36,V53,V70,V75,V76,V87,V91,V99,V126,V127,V128,V133,V145,V149,V152,V176,V177,V230,V257,V258,V285,V304,V306,V307,id_02,id_13,id_17,id_19,id_20,id_26,DeviceInfo,is_train_df,P_emaildomain_2,P_emaildomain_3,id_31_edge,id_30_android,TransactionAmt_decimal,weekday,hours,R_emaildomain_2_com,id_14_-420.0,M4_M0,M8_F,R_emaildomain_anonymous.com,M6_F,M5_F,P_emaildomain_hotmail.com,id_20_333.0,id_20_549.0,M7_F,id_32_24.0
0,2987000,0.0,0,4,12696,-1,50,1,42,1,215,77,NaN,NaN,0.452748,-56.367245,-35.609619,0.147989,-7.334560,-76.363113,-4.731055,-7.852631,-13.998543,-219.237869,T,M2,F,T,NaN,NaN,NaN,0.332354,NaN,NaN,0.311933,-0.696159,0.420110,0.371908,-0.400271,-0.477729,-5.658206,-442.928955,-6130.065430,-47.239407,93.224512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.146344,0.000325,-98.388420,-65.309341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,100.971710,0,0,-0.015990,-0.019621,-0.265431,-0.164929,-0.003047,-0.080853,0.437072,-0.025767,-0.006322,-0.000008,-0.390713,-0.016675
1,2987001,0.0,1,4,1726,304,50,2,2,1,225,77,gmail.com,NaN,-3.620259,-56.254463,-4.153918,-0.184928,-9.372200,-2.555614,-0.118743,-88.157257,-157.705399,-163.875885,NaN,M0,T,T,NaN,NaN,NaN,-0.726845,-0.738347,-0.600466,-0.845680,-0.669546,-0.728800,-0.656142,-0.179304,-0.418249,-7.665982,-29.248983,-1255.726807,10.947166,294.409943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.674220,0.007776,-1461.669067,325.092743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,com,NaN,NaN,NaN,-256.136840,0,0,-0.000335,0.000125,0.185291,-0.182496,0.005825,-0.163323,-0.526690,0.002152,-0.154146,-0.000008,-0.221700,-0.005720
2,2987002,0.0,2,4,3597,390,50,3,66,2,230,77,outlook.com,NaN,-4.552312,-57.456963,-5.875078,-0.109141,-13.440516,-8.605875,-3.505404,-48.127869,-70.123619,91.240555,T,M0,F,F,F,F,F,0.299800,0.297722,0.394673,0.299507,-0.521132,0.398391,0.318474,0.029664,-0.472105,-8.090937,-14.131649,-8801.166016,-127.531815,47.391029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.530957,0.000325,-206.973801,-566.554932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,com,NaN,NaN,NaN,-366.548200,0,0,0.001292,0.000063,0.431426,0.819766,0.029431,0.422267,0.758143,0.015687,-0.000358,-0.000008,0.765233,0.001808
3,2987003,0.0,3,4,16830,467,50,2,17,2,376,77,yahoo.com,NaN,-28.246696,-45.159729,-2.702492,0.607816,-11.248108,-48.973129,4.087717,-83.858826,-13.188324,-202.706940,NaN,M0,T,F,NaN,NaN,NaN,0.284040,0.588696,0.456558,0.216863,-0.823855,0.633004,0.214649,0.066225,-0.269714,0.613984,-16.667755,-7303.942383,875.265041,1494.660744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.829470,0.002005,109.774151,1643.618919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,com,NaN,NaN,NaN,-61.431190,0,0,0.007336,0.001207,0.319998,-0.146063,0.014364,0.494657,-0.807073,-0.036007,0.005367,-0.000008,-0.233716,-0.001090
4,2987004,0.0,4,1,3434,414,50,2,2,1,320,77,gmail.com,NaN,1.002734,-29.206808,0.961519,1.349431,-0.743106,-4.840970,5.024339,1.039412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.641993,-134.359344,-621.891052,-380.488739,-776.441528,55.91214,-0.341633,33.539467,-1.079679,-0.108888,-18.394489,-0.234223,-0.001191,0.114226,-0.017394,-2210.778564,-884.677673,-93479.875,NaN,166.0,542.0,144.0,NaN,SAMSUNG SM-G892A Build/NRD90M,1,com,NaN,False,True,-25.592527,0,0,-0.335331,-0.059717,-0.229276,0.016135,-0.174344,0.035477,0.056471,-0.084155,0.007734,0.005053,0.017360,-0.443237


In [6]:
master_df_df.shape

NameError: name 'master_df_df' is not defined

In [7]:
for col in master_df.select_dtypes(exclude='number').columns:
    master_df[col] = master_df[col].astype('category').cat.codes

In [8]:
'''
length_ones = len(master_df[master_df['isFraud']==1])
train_balanced = pd.concat([master_df[master_df['isFraud']==1], (master_df[master_df['isFraud']==0]).sample(length_ones)], axis=0)

#train_balanced = train_balanced.sample(10000)


X_train, X_test, y_train, y_test = train_test_split(
    train_balanced.drop(columns=['isFraud', 'TransactionID', 'TransactionDT']), train_balanced['isFraud'], 
    test_size=1/4, stratify =train_balanced['isFraud'],  random_state=0)

print(X_train.shape)
print(X_test.shape)

clf = XGBClassifier(max_depth=5, n_estimators=1000, verbosity=1)
clf.fit(X_train, y_train)
pred_prob = clf.predict_proba(X_test)
pred_prob[:, 1]
roc_score = roc_auc_score(y_test, pred_prob[:, 1])
print("roc_auc score %.4f" % roc_score)
xgboost.plot_importance(clf, max_num_features=20, importance_type='gain')
xgboost.plot_importance(clf, max_num_features=20, importance_type='weight')
'''

'\nlength_ones = len(master_df[master_df[\'isFraud\']==1])\ntrain_balanced = pd.concat([master_df[master_df[\'isFraud\']==1], (master_df[master_df[\'isFraud\']==0]).sample(length_ones)], axis=0)\n\n#train_balanced = train_balanced.sample(10000)\n\n\nX_train, X_test, y_train, y_test = train_test_split(\n    train_balanced.drop(columns=[\'isFraud\', \'TransactionID\', \'TransactionDT\']), train_balanced[\'isFraud\'], \n    test_size=1/4, stratify =train_balanced[\'isFraud\'],  random_state=0)\n\nprint(X_train.shape)\nprint(X_test.shape)\n\nclf = XGBClassifier(max_depth=5, n_estimators=1000, verbosity=1)\nclf.fit(X_train, y_train)\npred_prob = clf.predict_proba(X_test)\npred_prob[:, 1]\nroc_score = roc_auc_score(y_test, pred_prob[:, 1])\nprint("roc_auc score %.4f" % roc_score)\nxgboost.plot_importance(clf, max_num_features=20, importance_type=\'gain\')\nxgboost.plot_importance(clf, max_num_features=20, importance_type=\'weight\')\n'

In [9]:
train_balanced = master_df[master_df['isFraud'].notnull()]

temp_list_to_drop = []
temp_list_to_drop.extend(['isFraud', 'TransactionID', 'TransactionDT', 'is_train_df'])

print(train_balanced.shape)


clf = XGBClassifier(max_depth=50)
clf.fit(train_balanced.drop(columns=temp_list_to_drop), train_balanced['isFraud'])

(590540, 84)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=50,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [10]:
gc.collect()

19

In [11]:
# prepare submission
temp_list_to_drop = []
#temp_list_to_drop = list(cols_cat)
temp_list_to_drop.extend(['isFraud', 'TransactionID', 'TransactionDT'])
temp_list_to_include = list(set(master_df.columns).difference(set(temp_list_to_drop)))

temp_list_to_drop = []
#temp_list_to_drop = list(cols_cat)
temp_list_to_drop.extend(['isFraud', 'TransactionID', 'TransactionDT'])

temp_list_to_include =  list(train_balanced.drop(columns=temp_list_to_drop).columns)

temp_list_to_drop = []
#temp_list_to_drop = list(cols_cat)
temp_list_to_drop.extend(['isFraud', 'TransactionID', 'TransactionDT', 'is_train_df'])

counter_from = master_df.loc[master_df['is_train_df']==0, 'isFraud'].index[0]

len_master_df = len(master_df)

print(counter_from)
print(len_master_df)
print('start!!')
while counter_from < len_master_df:
    print(counter_from)
    counter_to = counter_from + 10000
    pred = pd.DataFrame()
    #print(len(master_df['isFraud'].loc[counter_from:counter_to]))
    #print(len(master_df.loc[counter_from:counter_to, [col for col in master_df.columns if col not in temp_list_to_drop]]))
    master_df['isFraud'].loc[counter_from:counter_to] =  clf.predict_proba(master_df.loc[counter_from:counter_to, [col for col in master_df.columns if col not in temp_list_to_drop]])[:, 1]
    
    counter_from += 10000
    gc.collect()
#print(temp_list_to_include)

590540
1097231
start!!
590540
600540
610540
620540
630540
640540
650540
660540
670540
680540
690540
700540
710540
720540
730540
740540
750540
760540
770540
780540
790540
800540
810540
820540
830540
840540
850540
860540
870540
880540
890540
900540
910540
920540
930540
940540
950540
960540
970540
980540
990540
1000540
1010540
1020540
1030540
1040540
1050540
1060540
1070540
1080540
1090540


In [12]:
#sample_submission.head()

In [13]:
counter_from = master_df.loc[master_df['is_train_df']==0, 'isFraud'].index[0]
submission = pd.DataFrame(master_df[['TransactionID', 'isFraud']].loc[counter_from:]).reset_index(drop = True)
submission.head()

,TransactionID,isFraud
0,3663549,0.000452
1,3663550,0.001100
2,3663551,0.001328
3,3663552,0.000681
4,3663553,0.001450


In [14]:
submission.describe()

,TransactionID,isFraud
count,5.066910e+05,506691.000000
mean,3.916894e+06,0.017426
std,1.462692e+05,0.102235
min,3.663549e+06,0.000061
25%,3.790222e+06,0.000621
50%,3.916894e+06,0.001331
75%,4.043566e+06,0.003378
max,4.170239e+06,0.999683


In [15]:
submission.to_csv('submission.csv', index=False)